In [1]:
%load_ext autoreload
%autoreload 2
%reload_ext autoreload

In [20]:
from datetime import datetime as dt

import tensorflow as tf
from tensorflow import keras

from module import DataLoader
import matplotlib.pyplot as plt
import pandas as pd

# 1. Data
- 해당의 데이터는 movielens 데이터이며, train_test_split 으로 나누어져 훈련용, 시험용 데이터로 나누어진다.
- generate_trainset, generate_testset 으로 인해 반환되는 값은 numpy array의 형식으로 X, y 로 반환된다.

In [23]:
loader = DataLoader()

X_train, y_train = loader.generate_trainset()
X_test, y_test = loader.generate_testset()

print("훈련 데이터 shape : {}, 훈련 라벨 갯수 : {}".format(X_train.shape, y_train.size))
print("테스트 데이터 shape : {}, 테스트 라벨 갯수 : {}".format(X_test.shape, y_test.size))

훈련 데이터 shape : (80003, 2), 훈련 라벨 갯수 : 80003
테스트 데이터 shape : (19259, 2), 테스트 라벨 갯수 : 19259


# 2. Generalized MF (Matrix Factorization)

## 1. Generate GMF Model Process

In [25]:
# 1. Setting Value 
num_users = loader.num_users
num_items = loader.num_items
latent_features = 8 # 임베딩 층이 만들어낼 잠재변수의 갯수
regs = [0, 0]

In [86]:
# 2. 입력 층 생성
user_input = keras.Input(shape=(1,), dtype='int32')
item_input = keras.Input(shape=(1,), dtype='int32')

In [132]:
# 3. embedding 층 생성
user_embedding = keras.layers.Embedding(
                    num_users,
                    latent_features,
                    embeddings_regularizer=keras.regularizers.l2(0),
                    name='user_emebedding')
item_embedding = keras.layers.Embedding(
                    num_items,
                    latent_features,
                    embeddings_regularizer=keras.regularizers.l2(0),
                    name='item_emebedding')

### 1. 중간점검
- embedding 층은 입력값에 대하여 어떠한 변화를 줄 때 사용된다. 주로 자연어 처리에서 one-hot encoding 을 진행할 때 사용 하는 층이다.
- 현재의 예제에서는 유저의 인덱스를 letent_features (잠재 의미)의 갯수를 가진 잠재 벡터로 만드는 역할을 하며, 여기에 적용되는 식은 l2 정규화를 사용한다.
    - 하지만 해당 단계 까지는 latent space라고 보아도 무방하다.

In [166]:
user_model = keras.Sequential()
item_model = keras.Sequential()

user_model.add(user_input)
user_model.add(user_embedding)

item_model.add(item_input)
item_model.add(item_embedding)

user_output = user_model(X_train[:, 0])
item_output = item_model(X_train[:, 1])

print("---- 사용자 ----")
print("입력 값: {}".format(X_train[0, 0]))
print("임베딩 층 출력 값: {}".format(user_output[0].numpy()))
print("훈련 데이터 shape: {}\n".format(user_output.shape))

print("---- 아이템 ----")
print("입력 값: {}".format(X_train[0, 1]))
print("임베딩 층 출력 값: {}".format(item_output[0].numpy()))
print("훈련 데이터 shape: {}".format(item_output.shape))

---- 사용자 ----
입력 값: 0
임베딩 층 출력 값: [[-0.04784346  0.0219305   0.02599366  0.01803069  0.00854274  0.00518177
   0.03042683  0.03014571]]
훈련 데이터 shape: (80003, 1, 8)

---- 아이템 ----
입력 값: 0
임베딩 층 출력 값: [[-0.03670434  0.00328875  0.04756906 -0.04486153 -0.04150504 -0.00676814
   0.01595345  0.03219468]]
훈련 데이터 shape: (80003, 1, 8)


In [167]:
# 4. Flatten 층
user_latent = keras.layers.Flatten()
item_latent = keras.layers.Flatten()

user_model.add(user_latent)
item_model.add(item_latent)

user_output = user_model(X_train[:, 0])
item_output = item_model(X_train[:, 1])

print("---- 사용자 ----")
print("입력 값: {}".format(X_train[0, 0]))
print("임베딩 층 출력 값: {}".format(user_output[0].numpy()))
print("훈련 데이터 shape: {}\n".format(user_output.shape))

print("---- 아이템 ----")
print("입력 값: {}".format(X_train[0, 1]))
print("임베딩 층 출력 값: {}".format(item_output[0].numpy()))
print("훈련 데이터 shape: {}".format(item_output.shape))

---- 사용자 ----
입력 값: 0
임베딩 층 출력 값: [-0.04784346  0.0219305   0.02599366  0.01803069  0.00854274  0.00518177
  0.03042683  0.03014571]
훈련 데이터 shape: (80003, 8)

---- 아이템 ----
입력 값: 0
임베딩 층 출력 값: [-0.03670434  0.00328875  0.04756906 -0.04486153 -0.04150504 -0.00676814
  0.01595345  0.03219468]
훈련 데이터 shape: (80003, 8)


In [169]:
# 5. concat with multiply
concat = keras.layers.Multiply()
merge_output = concat([user_output, item_output])

print("---- 사용자 X 아이템 ----")
print("입력 값: {}".format(X_train[0, 0], X_train[0, 1]))
print("임베딩 층 출력 값: {}".format(merge_output[0].numpy()))
print("훈련 데이터 shape: {}\n".format(merge_output.shape))

---- 사용자 X 아이템 ----
입력 값: 0
임베딩 층 출력 값: [ 1.7560625e-03  7.2123854e-05  1.2364938e-03 -8.0888445e-04
 -3.5456670e-04 -3.5070949e-05  4.8541278e-04  9.7053137e-04]
훈련 데이터 shape: (80003, 8)



In [175]:
# 6. Dense Layer
output_layer = keras.layers.Dense(1, kernel_initializer=keras.initializers.lecun_uniform(), name='output')
output = output_layer(merge_output)

print("---- GMF의 결과값은 사용자와 아이템 간의 선형적 관계를 나타낸다. ----")
print(output.numpy())

---- GMF의 결과값은 사용자와 아이템 간의 선형적 관계를 나타낸다. ----
[[-0.00051359]
 [-0.000428  ]
 [ 0.00065777]
 ...
 [ 0.00037375]
 [ 0.00054181]
 [ 0.00133845]]
